# param recovery for both model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna


# important directories

In [2]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

In [4]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})


In [5]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 1) or (my < your and choice == 0)):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [6]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,1067,uniform,894,win,3,9,3575,10.5,0,response,arrowdown,1,0,0,0.250,uniform
1,237,uniform,925,lose,2,1,903,10.0,1,response,arrowdown,1,0,1,0.125,uniform
2,231,uniform,973,win,8,2,1130,10.5,2,response,arrowup,1,0,1,0.125,uniform
3,602,uniform,768,win,7,2,537,11.0,3,response,arrowup,1,0,1,0.250,uniform
4,273,uniform,937,win,9,8,809,11.5,4,response,arrowup,1,0,1,0.000,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,567,high,787,win,9,7,1580,92.0,128,response,arrowup,4,0,0,0.000,mix
266,151,low,854,win,6,3,8541,92.5,89,response,arrowup,4,0,0,0.146,mix
267,178,low,863,win,8,5,370,93.0,77,response,arrowup,4,0,1,0.023,mix
268,328,high,770,lose,6,4,644,92.5,95,response,arrowdown,4,0,1,0.385,mix


In [7]:
df_colors_numbers[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,outcome
0,arrowup,0,9.5,10.5,"[[[-0.020376703070331587, -0.00073429560613807...",3,9,uniform,uniform,lose
1,arrowdown,0,9.0,10.0,"[[[-0.020376703070331587, -0.00073429560613807...",2,1,uniform,uniform,lose
2,arrowdown,1,8.5,10.5,"[[[-0.020376703070331587, -0.00073429560613807...",8,2,uniform,uniform,lose
3,arrowup,1,9.0,11.0,"[[[-0.020376703070331587, -0.00073429560613807...",7,2,uniform,uniform,lose
4,arrowup,1,9.5,11.5,"[[[-0.020376703070331587, -0.00073429560613807...",9,8,uniform,uniform,lose
...,...,...,...,...,...,...,...,...,...,...
265,arrowup,1,69.0,92.0,"[[[0.4887777287028654, -0.0007342956061380753]...",9,7,high,mix,lose
266,arrowup,1,69.5,92.5,"[[[0.4887777287028654, -0.0007342956061380753]...",6,3,low,mix,lose
267,arrowdown,1,69.0,93.0,"[[[0.4887777287028654, -0.0007342956061380753]...",8,5,low,mix,lose
268,arrowdown,0,68.5,92.5,"[[[0.4887777287028654, -0.0007342956061380753]...",6,4,high,mix,lose


In [8]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.xlsx')]

In [9]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

# policy_table = percentage_matrix 


Q_table_init = np.random.normal(0, 0.01, (len(card_numbers), len(distributions_map), len(actions)))


Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [10]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["model_choices"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [11]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 0.8
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [12]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()
    
    df_all = df_all[df_all['outcome'].str.lower() != 'na'].reset_index(drop=True)

    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)

    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choices.append(np.random.choice([1, 0], p=[p_arrowup, p_arrow_down]))

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    print(best_alpha)
    print(best_beta)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)





[I 2025-06-10 14:20:54,561] A new study created in memory with name: no-name-17bea27c-6d76-4665-8c06-5ef5008b0097


Processing participant 1 of 35


[I 2025-06-10 14:20:55,533] Trial 1 finished with value: 228.6843085473256 and parameters: {'alpha': 0.18434687873195357, 'beta': 0.7015220644847112}. Best is trial 1 with value: 228.6843085473256.
[I 2025-06-10 14:20:55,716] Trial 7 finished with value: 483.97546384545944 and parameters: {'alpha': 0.3117178777262553, 'beta': 0.14520411961162752}. Best is trial 1 with value: 228.6843085473256.
[I 2025-06-10 14:20:55,730] Trial 15 finished with value: 891.0027253891626 and parameters: {'alpha': 0.43712859197249215, 'beta': 0.015013147733925966}. Best is trial 1 with value: 228.6843085473256.
[I 2025-06-10 14:20:55,737] Trial 8 finished with value: 238.0860815174496 and parameters: {'alpha': 0.5186119578440687, 'beta': 0.6549474316005099}. Best is trial 1 with value: 228.6843085473256.
[I 2025-06-10 14:20:55,745] Trial 23 finished with value: 310.6741399678423 and parameters: {'alpha': 0.7114532632493847, 'beta': 0.404733049446579}. Best is trial 1 with value: 228.6843085473256.
[I 2025-

0.010474496130869211
0.7976628962332675
Processing participant 2 of 35


[I 2025-06-10 14:21:19,770] Trial 7 finished with value: 347.6333356639964 and parameters: {'alpha': 0.9004383663902441, 'beta': 0.36331606784732473}. Best is trial 7 with value: 347.6333356639964.
[I 2025-06-10 14:21:19,813] Trial 29 finished with value: 232.75571906065596 and parameters: {'alpha': 0.049326567024320186, 'beta': 0.7169742039254513}. Best is trial 27 with value: 220.0150006495076.
[I 2025-06-10 14:21:19,813] Trial 4 finished with value: 443.1534255287945 and parameters: {'alpha': 0.7569843429520365, 'beta': 0.2153214193988177}. Best is trial 27 with value: 220.0150006495076.
[I 2025-06-10 14:21:19,813] Trial 12 finished with value: 434.59034770301145 and parameters: {'alpha': 0.9910310458451203, 'beta': 0.2254821735224528}. Best is trial 27 with value: 220.0150006495076.
[I 2025-06-10 14:21:19,813] Trial 27 finished with value: 220.0150006495076 and parameters: {'alpha': 0.8534937008663114, 'beta': 0.7835711428478593}. Best is trial 27 with value: 220.0150006495076.
[I 

...

0.010538371080912233
0.7997887422173594
Processing participant 34 of 35


[I 2025-06-10 14:33:41,254] Trial 13 finished with value: 315.5202719759281 and parameters: {'alpha': 0.42939177885879576, 'beta': 0.3427263258688624}. Best is trial 13 with value: 315.5202719759281.
[I 2025-06-10 14:33:41,271] Trial 0 finished with value: 244.07338052214718 and parameters: {'alpha': 0.7208559324476019, 'beta': 0.5746354873873971}. Best is trial 0 with value: 244.07338052214718.
[I 2025-06-10 14:33:41,287] Trial 17 finished with value: 349.24790509881495 and parameters: {'alpha': 0.7724963110671555, 'beta': 0.27350762248318267}. Best is trial 0 with value: 244.07338052214718.
[I 2025-06-10 14:33:41,322] Trial 2 finished with value: 214.0774081853556 and parameters: {'alpha': 0.012865211008307122, 'beta': 0.6866119391111597}. Best is trial 2 with value: 214.0774081853556.
[I 2025-06-10 14:33:41,328] Trial 5 finished with value: 345.97289325531455 and parameters: {'alpha': 0.901875452376439, 'beta': 0.2794790013398474}. Best is trial 2 with value: 214.0774081853556.
[I 2

0.010020506763484338
0.7994540228433232
Processing participant 35 of 35


[I 2025-06-10 14:34:03,872] Trial 0 finished with value: 328.9298386493865 and parameters: {'alpha': 0.8422451184997751, 'beta': 0.2627148709112464}. Best is trial 0 with value: 328.9298386493865.
[I 2025-06-10 14:34:03,939] Trial 19 finished with value: 276.89680576140495 and parameters: {'alpha': 0.8927454114824074, 'beta': 0.3900541626943583}. Best is trial 19 with value: 276.89680576140495.
[I 2025-06-10 14:34:04,006] Trial 12 finished with value: 308.9104629824873 and parameters: {'alpha': 0.5977025410916492, 'beta': 0.3047581267890441}. Best is trial 19 with value: 276.89680576140495.
[I 2025-06-10 14:34:04,056] Trial 29 finished with value: 223.9080410399783 and parameters: {'alpha': 0.33737639995383994, 'beta': 0.616955318064328}. Best is trial 29 with value: 223.9080410399783.
[I 2025-06-10 14:34:04,074] Trial 2 finished with value: 386.0331218598744 and parameters: {'alpha': 0.855252958328642, 'beta': 0.17469482989470836}. Best is trial 29 with value: 223.9080410399783.
[I 20

0.2063547272419023
0.7999877258139938


# now saving the model evaluation values

In [13]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_greedy.csv")
df_models_evaluation.to_csv(file_path, index=False)